The shell is the application either jupyterlab or jupyter notebook, the kernel determines the programming language.  Below we design a just jupyter kernel that can be installed using 

    !pidgy kernel install

In [2]:
    
    import jupyter_client, IPython, ipykernel, pidgy, traitlets, pidgy, traitlets, ipykernel.kernelspec, pathlib

In [7]:

    class pidgyInteractiveShell(ipykernel.zmqshell.ZMQInteractiveShell):
        """Configure a native `pidgy` `IPython.InteractiveShell`"""        
        enable_html_pager = traitlets.Bool(True)
        def __init__(self,*args, **kwargs):
            super().__init__(*args, **kwargs)
            pidgy.extension.load_ipython_extension(self)
            
    class pidgyKernel(ipykernel.ipkernel.IPythonKernel):
        shell_class = traitlets.Type(pidgyInteractiveShell)

In [5]:
    def install(kernel_name='pidgy',
        user=True,
        replace=None,
        prefix=None
    ):
        return ipykernel.kernelspec.KernelSpecManager().install_kernel_spec(
            str(pathlib.Path(globals().get('__file__', pathlib.Path('spec'))).parent/'spec'), kernel_name=kernel_name,
            user=user, replace=replace, prefix=prefix)

    def install(kernel_name='pidgy',
        user=True,
        replace=None,
        prefix=None
    ):
        return ipykernel.kernelspec.KernelSpecManager().install_kernel_spec(
            str(pathlib.Path(globals().get('__file__', pathlib.Path('spec'))).parent/'spec'), kernel_name=kernel_name,
            user=user, replace=replace, prefix=prefix)

In [6]:
    def uninstall(kernel_name='pidgy',):
        ipykernel.kernelspec.KernelSpecManager().remove_kernel_spec(kernel_name)

    def uninstall(kernel_name='pidgy',):
        ipykernel.kernelspec.KernelSpecManager().remove_kernel_spec(kernel_name)